# Markov decision process(MDP)

에이전트에게 완전한 문제를 제시하는 환경을 마르코프 결정 과정(Markov decision process, MDP)라고 한다. <br>
이러한 환경은 주어진 환경에서의 보상과 상태 전이를 제공하는 데 그치지 않고, 보상 역시 환경의 상태와 에이전트가 자신의 상태에서 취하는 액션에 의해 좌우

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt

In [2]:
# 오픈AI에서 특정 환경을 불러오는 코드
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [20]:
gamma = 0.99

def discount_rewards(r):
    # 보상의 1D 실수 배열을 취해서 할인된 보상을 계산
    discounted_r = np.zeros_like(r)
    running_add = 0
    
    for t in range(r.size-1, -1, -1):
        running_add = running_add*gamma+r[t]
        discounted_r[t] = running_add
    
    return discounted_r

class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # 네트워크의 피드포워드 부분, 에이전트는 상태를 받아서 액션을 출력
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        # 히든레이어 설정
        hidden = slim.fully_connected(inputs=self.state_in, num_outputs=h_size, activation_fn=tf.nn.relu, biases_initializer=None)
        # 출력레이어 설정
        self.output = slim.fully_connected(inputs=hidden, num_outputs=a_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        # 출력레이어에서 값이 가장 큰 값의 인덱스 선택
        self.chosen_action = tf.argmax(input=self.output, axis=1)
        
        # 학습 과정 구현, 비용을 계산하기 위해 보상과 액션을 네트워크에 피드하고, 이를 통해 네트워크 업데이트
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indexes = tf.range(start=0, limit=tf.shape(input=self.output)[0])*tf.shape(input=self.output)[1]+self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        
        # 평균값 줄이기
        self.loss = -tf.reduce_mean(input_tensor=tf.log(self.responsible_outputs)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx, ele in enumerate(tvars):
            placeholder = tf.placeholder(dtype=tf.float32, name=str(idx)+"_holder")
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(ys=self.loss, xs=tvars)
        
        optimizer =tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))